<a href="https://colab.research.google.com/github/antopeza/Final-Project-Brainster/blob/main/Doc2Vec_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from numpy import random
from numpy import vstack
import matplotlib.pyplot as plt
import os 

import re

import string
from string import punctuation

from sklearn.model_selection import ShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingGridSearchCV

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import sklearn.metrics as metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import xgboost as xgb

import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer
from nltk.corpus import wordnet, stopwords
from nltk import pos_tag, word_tokenize

import gensim
from gensim.parsing.preprocessing import remove_stopwords

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Embedding, LSTM, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

from imblearn.over_sampling import SMOTE
from collections import Counter

#from scikeras.wrappers import KerasClassifier

import pickle
from pickle import dump
from pickle import load

from imblearn.over_sampling import SMOTE
from collections import Counter

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive 
drive.mount('/drive')

Mounted at /drive


In [4]:
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("averaged_perceptron_tagger")
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [5]:
stops = set(stopwords.words('english'))
#print(stops)

In [6]:
stemmer =  PorterStemmer()
lemmer = WordNetLemmatizer()

In [7]:
#df_raw = pd.read_csv(path)
df_raw = pd.read_csv('/drive/My Drive/MACHINE LEARNING/FINAL PROJECT/train.csv')

In [ ]:
# print('shape of df  = ', df_raw.shape)
# print()
# df_raw.head(2)

shape of df  =  (404290, 6)



,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0


In [ ]:
#df_raw[df_raw.isnull().any(axis=1)]

,id,qid1,qid2,question1,question2,is_duplicate
105780,105780,174363,174364,How can I develop android app?,NaN,0
201841,201841,303951,174364,How can I create an Android app?,NaN,0
363362,363362,493340,493341,NaN,My Chinese name is Haichao Yu. What English na...,0


In [8]:
df_raw = df_raw.fillna("")
df_raw.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [9]:
# randomly selected samples from df for the new df to perform operations
df = df_raw.sample(50000)
df = df[['question1', 'question2', 'is_duplicate']]
print(df.shape)

(50000, 3)


In [ ]:
#df_copy = df.copy()

# Clean Data

In [10]:
def get_wordnet_pos(tag):
    if tag.startswith("N"):
        return wordnet.NOUN
    if tag.startswith("J"):
        return wordnet.ADJ
    if tag.startswith("V"):
        return wordnet.VERB
    if tag.startswith("R"):
        return wordnet.ADV
    return wordnet.NOUN

def clean_sentence(sentence, stopwords = False, lemmatize = False, stem = False):
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence)  # removing all caracters that are not alpha numeric
    #sentence = re.sub(r'\s{2,}', ' ', sentence
   
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))  # remove punctuation
    
    if stopwords:
        sentence = remove_stopwords(sentence)  
        
    if lemmatize:
        nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
        wn_tagged = map(lambda x: (x[0], get_wordnet_pos(x[1])), nltk_tagged)
        res_words = []
        for word, tag in wn_tagged:
            if tag is None:
                res_words.append(word)
            else :
                res_words.append(lemmer.lemmatize(word, tag))
        sentence = " ".join(res_words)
        
    if stem:
        sent_stemmed = ''
        for word in sentence.split():
            sent_stemmed += ' '+ stemmer.stem(word)
        sentence = sent_stemmed
   
    return sentence

def get_cleaned_senteces(df, stopwords = False, lemmatize = False, stem = False):
    #sents1 = dfq[['question1']]  
    #sents2 = dfq[['question2']]
    cleaned_sentences1 = []
    cleaned_sentences2 = []
    
    for index, row in df.iterrows():
        # print (index, row)
        cleaned1 = clean_sentence(row['question1'], stopwords, lemmatize, stem) 
        cleaned2 = clean_sentence(row['question2'], stopwords, lemmatize, stem)

        cleaned_sentences1.append(cleaned1)
        cleaned_sentences2.append(cleaned2)
    return cleaned_sentences1, cleaned_sentences2

In [11]:
def to_df(q1cleaned, q2cleaned, df):
    '''cleaned sentences to df'''
    X_temp1 = pd.DataFrame()
    X_temp2 = pd.DataFrame()
    X_temp1['q1'] = pd.DataFrame(q1cleaned, index = df.index)
    X_temp2['q2'] = pd.DataFrame(q2cleaned, index = df.index)
    X_temp = pd.concat([X_temp1, X_temp2], axis = 1)
    X_temp['is_duplicate'] = df_raw['is_duplicate']
    return X_temp

#### With Stopwords, lemmatized

In [12]:
# With stopwords lemmatized, not stemmed
q1_sw_lemma, q2_sw_lemma = get_cleaned_senteces(df, stopwords = False, lemmatize = True, stem = False)
assert len(q1_sw_lemma) == len(q2_sw_lemma)
df_sw_lemma = to_df(q1_sw_lemma, q2_sw_lemma, df)
df_sw_lemma.shape

(50000, 3)

#### With Stopwords, stemmed

In [13]:
q1_sw_stem, q2_sw_stem = get_cleaned_senteces(df, stopwords = False, lemmatize = False, stem = True)
assert len(q1_sw_stem) == len(q2_sw_stem)
df_sw_stem = to_df(q1_sw_stem, q2_sw_stem, df)
df_sw_stem.shape

(50000, 3)

#### With stopwords, not lemmatized nor stemmed

In [14]:
q1_sw, q2_sw = get_cleaned_senteces(df, stopwords = False, lemmatize = False, stem = False)
assert len(q1_sw) == len(q2_sw)
df_sw = to_df(q1_sw, q2_sw, df)
df_sw.shape

(50000, 3)

#### Without Stopwords, lemmatized


In [15]:
q1_lemma, q2_lemma = get_cleaned_senteces(df, stopwords = True, lemmatize = True, stem = False)
assert len(q1_lemma) == len(q2_lemma)
df_lemma = to_df(q1_lemma, q2_lemma, df)
df_lemma.shape

(50000, 3)

#### Without Stopwords, stemmed

In [22]:
q1_stem, q2_stem = get_cleaned_senteces(df, stopwords = True, lemmatize = False, stem = True)
assert len(q1_stem) == len(q2_stem)
df_stem = to_df(q1_stem, q2_stem, df)
df_stem.shape

(50000, 3)

#### Without Stopwords, not lemmatized nor stemmed

In [16]:
q1_out, q2_out = get_cleaned_senteces(df, stopwords = True, lemmatize = False, stem = False)
assert len(q1_out) == len(q2_out)
df_out = to_df(q1_out, q2_out, df)
df_out.shape

(50000, 3)

In [ ]:
display ('With stopwords, without lemma and stem', df_sw.iloc[[22]])
display ('With stopwords, lemmatized and without stem', df_sw_lemma.iloc[[22]])
display ('With stopwords, stemmed and without lemma', df_sw_stem.iloc[[22]])
display ('Without stopwords, lemma and stem', df_out.iloc[[22]])
display ('Without stopwords and stem, lemmatized', df_lemma.iloc[[22]])
display ('Without stopwords and lemma, stemmed', df_stem.iloc[[22]])

'With stopwords, without lemma and stem'

,q1,q2,is_duplicate
272872,what does it feel like to be on quora,what does it feel like to post on quora,1


'With stopwords, lemmatized and without stem'

,q1,q2,is_duplicate
272872,what do it feel like to be on quora,what do it feel like to post on quora,1


'With stopwords, stemmed and without lemma'

...

,q1,q2,is_duplicate
272872,feel like quora,feel like post quora,1


'Without stopwords and stem, lemmatized'

,q1,q2,is_duplicate
272872,feel like quora,feel like post quora,1


'Without stopwords and lemma, stemmed'

,q1,q2,is_duplicate
272872,feel like quora,feel like post quora,1


In [ ]:
# print('Value counts:\n', X_sw_train['is_duplicate'].value_counts())
# print('\nPercent of labels value:\n', X_sw_train['is_duplicate'].value_counts()/X_sw_train['is_duplicate'].count()*100)
# X_sw_train['is_duplicate'].value_counts().plot(kind = 'bar')
# plt.title('\nDistribution of train labels', color = 'purple', size = 12)
# plt.show()

In [ ]:
# print('Value counts:\n', X_sw_test['is_duplicate'].value_counts())
# print('\nPercent of labels value:\n', X_sw_test['is_duplicate'].value_counts()/X_sw_test['is_duplicate'].count()*100)
# X_sw_test['is_duplicate'].value_counts().plot(kind = 'bar')
# plt.title('\nDistribution of test labels', color = 'purple', size = 12)
# plt.show()

In [ ]:
# display (X_sw_train.head(2))
# display(X_sw_test.head(2))
# display(df_clean.head(2))

In [ ]:
#X_sw_test[X_sw_test.isnull().any(axis=1)]

### TOKENIZATION

In [ ]:
def tokenization(text):
    tokens = re.split(' ',text)
    return tokens
def token(text):
    text = text.apply(lambda x: tokenization(x))
    return text

In [ ]:
#Tokenize - lemmatized data without stopwords
df_lemma['q1'] = df_lemma['q1'].pipe(token) 
df_lemma['q2'] = df_lemma['q2'].pipe(token) 
df_lemma.head(2)

,q1,q2,is_duplicate
296968,"[indian, lamborghini, usa, legally]","[study, usa, indian, good, idea]",0
236824,"[learn, english]","[learn, english]",0


In [ ]:
# Tokenize - lemmatized data with stopwords
df_sw_lemma['q1'] = df_sw_lemma['q1'].pipe(token) 
df_sw_lemma['q2'] = df_sw_lemma['q2'].pipe(token) 
df_sw_lemma.head(2)

In [ ]:
# Tokenize - Stemmed data with stopwords
df_sw_stem['q1'] = df_sw_stem['q1'].pipe(token) 
df_sw_stem['q2'] = df_sw_stem['q2'].pipe(token) 
df_sw_stem.head(2)

,q1,q2,is_duplicate
11965,"[, what, is, the, list, of, job, titl, at, rsm...","[, which, countri, is, the, best, to, work, in...",0
342404,"[, should, peopl, over, 94, not, be, allow, to...","[, should, peopl, over, 97, not, be, allow, to...",1


In [ ]:
# Tokenize - Stemmed data without stopwords
df_stem['q1'] = df_stem['q1'].pipe(token) 
df_stem['q2'] = df_stem['q2'].pipe(token) 
df_stem.head(2)

,q1,q2,is_duplicate
327246,"[, can, i, regist, my, startup, compani, in, t...","[, can, i, start, a, compani, in, the, us, wit...",1
26328,"[, what, is, a, good, entrylevel, job, in, the...","[, what, are, the, best, entri, level, job, in...",0


In [ ]:
# Tokenize - With stopwords, not lemmatized nor stemmed
df_sw['q1'] = df_sw['q1'].pipe(token) 
df_sw['q2'] = df_sw['q2'].pipe(token) 
df_stem.head(2)

In [ ]:
# Tokenize - Without stopwords, not lemmatized nor stemmed
df_out['q1'] = df_out['q1'].pipe(token) 
df_out['q2'] = df_out['q2'].pipe(token) 
df_out.head(2)

NameError: ignored

In [ ]:
df_sw_lemma.head(2)

,q1,q2,is_duplicate
82361,who invent wash machine,who invent the washing machine,1
12785,be hack a crime when someone do it to earn mon...,what do pakistani girl look for in a guy objec...,0


# Doc2Vec function

In [17]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
def doc2vec(df):
  q1 = [TaggedDocument(d, [i]) for i, d in enumerate(df['q1'])]
  q2 = [TaggedDocument(d, [i]) for i, d in enumerate(df['q2'])]
  return q1, q2

In [117]:
def model_doc2vec(q1, dm, vector, window, epochs):
  q1 = Doc2Vec(q1, dm = dm, vector_size = vector, window = window, min_count = 2, epochs = epochs)
  return q1

### Run Doc2Vec Function

In [19]:
#lemmatized, without stopwords
q1_lemma_d2v, q2_lemma_d2v = doc2vec(df_lemma)
assert len(q1_lemma_d2v) == len(q2_lemma_d2v)

In [ ]:
#q1_lemma_d2v

In [20]:
#lemmatized with stopwords
q1_sw_lemma_d2v, q2_sw_lemma_d2v = doc2vec(df_sw_lemma)
assert len(q1_sw_lemma_d2v) == len(q2_sw_lemma_d2v)

In [23]:
#Stemmed without stopwords
q1_stem_d2v, q2_stem_d2v = doc2vec(df_stem)
assert len(q1_stem_d2v) == len(q2_stem_d2v)

In [24]:
#Stemmed with stopwords
q1_sw_stem_d2v, q2_sw_stem_d2v = doc2vec(df_sw_stem)
assert len(q1_sw_stem_d2v) == len(q2_sw_stem_d2v)

In [93]:
#With Stopwords, not lemmatized, not stemmed
q1_sw_d2v, q2_sw_d2v = doc2vec(df_sw)
assert len(q1_sw_d2v) == len(q2_sw_d2v)

In [26]:
#Without stopwords, not lemmatized, not stemmed 
q1_out_d2v, q2_out_d2v = doc2vec(df_out)
assert len(q1_out_d2v) == len(q2_out_d2v)

### Activating Doc2Vec model

In [150]:
#PARAMETER FOR DOC2VEC MODEL 
dm = 1
window = 3
vector = 100
epochs = 50

In [151]:
#lemmatized, without stopwords
X_q1_lemma_d2v = model_doc2vec(q1_lemma_d2v, dm, vector, window, epochs)
X_q2_lemma_d2v = model_doc2vec(q2_lemma_d2v, dm,  vector, window, epochs)

In [72]:
#lemmatized, with stopwords
X_q1_sw_lemma_d2v = model_doc2vec(q1_sw_lemma_d2v, vector, epochs)
X_q2_sw_lemma_d2v = model_doc2vec(q2_sw_lemma_d2v, vector, epochs)

In [73]:
#Stemmed, without stopwords
X_q1_stem_d2v = model_doc2vec(q1_stem_d2v, vector,  epochs)
X_q2_stem_d2v = model_doc2vec(q2_stem_d2v, vector,  epochs)

In [69]:
#Stemmed, with stopwords
X_q1_sw_stem_d2v = model_doc2vec(q1_sw_stem_d2v, vector, epochs)
X_q2_sw_stem_d2v = model_doc2vec(q2_sw_stem_d2v, vector, epochs)

In [94]:
#With Stopwords, not lemmatized, not stemmed
X_q1_sw_d2v = model_doc2vec(q1_sw_d2v, vector, epochs)
X_q2_sw_d2v = model_doc2vec(q2_sw_d2v, vector, epochs)

In [74]:
#Without stopwords, not lemmatized, not stemmed 
X_q1_out_d2v = model_doc2vec(q1_out_d2v, vector, epochs)
X_q2_out_d2v = model_doc2vec(q2_out_d2v, vector, epochs)


#### Convert to array 

In [152]:
# Lemmatized, without stopwords, not stemmed

size = len(df) - 1
arr_q1_lemma = X_q1_lemma_d2v[0] 

for i in range(size):
  arrT = X_q1_lemma_d2v[i+1]
  arr_q1_lemma = np.vstack([arr_q1_lemma, arrT])

arr_q2_lemma = X_q2_lemma_d2v[0] 

for i in range(size):
  arrT = X_q2_lemma_d2v[i+1]
  arr_q2_lemma = np.vstack([arr_q2_lemma, arrT])

assert arr_q1_lemma.shape == arr_q2_lemma.shape

#arr_q1_lemma = create_arrays(X_q1_lemma_d2v)
#arr_q2_lemma = create_arrays(X_q2_lemma_d2v)

In [75]:
# Lemmatized, with stopwords, not stemmed

size = len(df) - 1
arr_q1_sw_lemma = X_q1_sw_lemma_d2v[0] 

for i in range(size):
  arrT = X_q1_sw_lemma_d2v[i+1]
  arr_q1_sw_lemma = np.vstack([arr_q1_sw_lemma, arrT])

arr_q2_sw_lemma = X_q2_sw_lemma_d2v[0] 

for i in range(size):
  arrT = X_q2_sw_lemma_d2v[i+1]
  arr_q2_sw_lemma = np.vstack([arr_q2_sw_lemma, arrT])

assert arr_q1_sw_lemma.shape == arr_q2_sw_lemma.shape

# arr_q1_sw_lemma = create_arrays(X_q1_sw_lemma_d2v)
# arr_q2_sw_lemma = create_arrays(X_q2_sw_lemma_d2v) 

In [76]:
# Stemmed, without stopwords, not lemmatized

size = len(df) - 1
arr_q1_stem = X_q1_stem_d2v[0] 

for i in range(size):
  arrT = X_q1_stem_d2v[i+1]
  arr_q1_stem = np.vstack([arr_q1_stem, arrT])

arr_q2_stem = X_q2_stem_d2v[0] 

for i in range(size):
  arrT = X_q2_stem_d2v[i+1]
  arr_q2_stem = np.vstack([arr_q2_stem, arrT])

assert arr_q1_stem.shape == arr_q2_stem.shape

# arr_q1_stem = create_arrays(X_q1_stem_d2v)
# arr_q2_stem = create_arrays(X_q2_stem_d2v)

In [77]:
# Stemmed, with stopwords, not lemmatized 

size = len(df) - 1
arr_q1_sw_stem = X_q1_sw_stem_d2v[0] 

for i in range(size):
  arrT = X_q1_sw_stem_d2v[i+1]
  arr_q1_sw_stem = np.vstack([arr_q1_sw_stem, arrT])

arr_q2_sw_stem = X_q2_sw_stem_d2v[0] 

for i in range(size):
  arrT = X_q2_sw_stem_d2v[i+1]
  arr_q2_sw_stem = np.vstack([arr_q2_sw_stem, arrT])

assert arr_q1_sw_stem.shape == arr_q2_sw_stem.shape

# arr_q1_sw_stem = create_arrays(X_q1_sw_stem_d2v)
# arr_q2_sw_stem = create_arrays(X_q2_sw_stem_d2v)

KeyboardInterrupt: ignored

In [ ]:
# Without stopwords, not lemmatized, not stemmed
size = len(df) - 1
arr_q1_out = X_q1_out_d2v[0] 

for i in range(size):
  arrT = X_q1_out_d2v[i+1]
  arr_q1_out = np.vstack([arr_q1_out, arrT])

arr_q2_out = X_q2_out_d2v[0] 

for i in range(size):
  arrT = X_q2_out_d2v[i+1]
  arr_q2_out = np.vstack([arr_q2_out, arrT])

assert arr_q1_out.shape == arr_q2_out.shape

In [95]:
# With stopword, not lemmatized, not stemmed

size = len(df) - 1
arr_q1_sw = X_q1_sw_d2v[0] 

for i in range(size):
  arrT = X_q1_sw_d2v[i+1]
  arr_q1_sw = np.vstack([arr_q1_sw, arrT])

arr_q2_sw = X_q2_sw_d2v[0] 

for i in range(size):
  arrT = X_q2_sw_d2v[i+1]
  arr_q2_sw = np.vstack([arr_q2_sw, arrT])

assert arr_q1_sw.shape == arr_q2_sw.shape

### Convert to dataframe 

In [33]:
def df_arr(arr_q1, arr_q2, df):
  temp1 = pd.DataFrame(arr_q1)
  temp2 = pd.DataFrame(arr_q2)
  X_temp = pd.concat([temp1, temp2], axis=1)
  y = df['is_duplicate']
  y = np.array(y)
  y_df = pd.DataFrame(y)
  df_X = X_temp.join(y_df,  lsuffix='_left', rsuffix='_right')
  #X_temp['is_duplicate'] = df['is_duplicate']
  return df_X

In [153]:
# Dataframe - Lemmatized without stopwords 
lemma_df = df_arr(arr_q1_lemma, arr_q2_lemma, df)

In [ ]:
# Dataframe - Lemmatized with stopwords 
sw_lemma_df = df_arr(arr_q1_sw_lemma, arr_q2_sw_lemma, df)

In [78]:
# Dataframe - Stemmed without stopwords 
stem_df = df_arr(arr_q1_stem, arr_q2_stem, df)

In [ ]:
# Dataframe - Lemmatized with stopwords 
sw_stem_df = df_arr(arr_q1_sw_stem, arr_q2_sw_stem, df)

In [96]:
# Dataframe - With stopwords, not lemmatized, not stemmed
sw_df = df_arr(arr_q1_sw, arr_q2_sw, df)

In [ ]:
# Dataframe - Without stopwords, not lemmatized, not stemmed
out_df = df_arr(arr_q1_out, arr_q2_out, df)

### Split Data

In [123]:
# Lemmatized with stopwords 
y_sw_lemma= sw_lemma_df['0_right']
X_train_sw_lemma, X_test_sw_lemma, y_train_sw_lemma, y_test_sw_lemma = train_test_split(sw_lemma_df, y_sw_lemma, stratify = y_sw_lemma, test_size = 0.15, random_state = 42)

NameError: ignored

In [154]:
# Lemmatized without stopwords 
y_lemma= lemma_df['0_right']
X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma = train_test_split(lemma_df, y_lemma, stratify = y_lemma, test_size = 0.15, random_state = 42)

In [ ]:
# Stemmed with stopwords 
y_sw_stem= sw_stem_df['0_right']
X_train_sw_stem, X_test_sw_stem, y_train_sw_stem, y_test_sw_stem = train_test_split(sw_stem_df, y_sw_stem, stratify = y_sw_stem, test_size = 0.15, random_state = 42)

In [79]:
# Stemmed without stopwords 
y_stem= stem_df['0_right']
X_train_stem, X_test_stem, y_train_stem, y_test_stem = train_test_split(stem_df, y_stem, stratify = y_stem, test_size = 0.15, random_state = 42)

In [97]:
# With Stopwords, not lemmatized, not stemmed 
y_sw= sw_df['0_right']
X_train_sw, X_test_sw, y_train_sw, y_test_sw = train_test_split(sw_df, y_sw, stratify = y_sw, test_size = 0.15, random_state = 42)

In [ ]:
# Without Stopwords, not lemmatized, not stemmed 
y_out= out_df['0_right']
X_train_out, X_test_out, y_train_out, y_test_out = train_test_split(out_df, y_out, stratify = y_out, test_size = 0.15, random_state = 42)

# MODELS

In [ ]:
# Creating DataFrame for Evaluations
result_se = pd.DataFrame(columns = ['Model Name', 'Accuracy', 'Precision', 'Recall', 'F1_score', 'Log_loss'])
result_se.head()

,Model Name,Accuracy,Precision,Recall,F1_score,Log_loss


In [ ]:
def res(model, acc, precision, recall, f1_score, log_loss):
    result_se.loc[len(result_se)]   = [model, acc, precision, recall, f1_score, log_loss]
    pd.set_option('display.max_rows', None)
    display (result_se)
    return result_se

### Random Forest

In [36]:
def rf_clf(X_train, X_test, y_train, y_test):
    '''Parameter tuning for Random Forest Classifier and model fit'''
    
    #splits = ShuffleSplit(n_splits = 1, test_size = .2, random_state = 0)

    rf_clf = RandomForestClassifier(random_state = 0)
    rf_param_grid = { "max_depth": [4, None],
               "min_samples_split": [ 20, 40]
              }
    
    
    rf_search = HalvingGridSearchCV(rf_clf, rf_param_grid, scoring='accuracy', resource='n_estimators', max_resources=5,              
                              random_state=0).fit(X_train, y_train)
    
    rf_model = rf_search.best_estimator_

    y_pred = rf_model.predict(X_test)
    
    log_loss = metrics.log_loss(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1_score = metrics.f1_score(y_test, y_pred)
    print (rf_model)
    #print('log loss', rf_loss_bow, '\nacc', rf_acc_bow)
    return accuracy, precision, recall, f1_score, log_loss, y_pred, rf_model

## Logistic Regression

In [105]:
def lr_clf(X_train, X_test, y_train, y_test):
    '''Parameter tuning for Logistic Regression Classifier and model fit'''
    
    splits = ShuffleSplit(n_splits = 1, test_size = .2, random_state = 0)
    #lr_param_grid = {'C':[100, 10]}     
    lr_clf = LogisticRegression(C=0.001, random_state = 0)
   
    #lr_search = HalvingGridSearchCV(lr_clf, lr_param_grid, scoring = 'accuracy')
    lr_clf.fit(X_train, y_train)
    
    #lr_model = lr_search.best_estimator_
    
    y_pred = lr_clf.predict(X_test)
    
    log_loss = metrics.log_loss(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1_score = metrics.f1_score(y_test, y_pred)
    #print (lr_model)
    #print('log loss', rf_loss_bow, '\nacc', rf_acc_bow)
    return accuracy, precision, recall, f1_score, log_loss, y_pred

# Support Vector Classification - SVC

In [101]:
def svc_clf(X_train, X_test, y_train, y_test):
    '''Parameter tuning for XGB Classifier and model fit'''
   
    splits = ShuffleSplit(n_splits = 1, test_size = .2, random_state = 0)
    
    #svc_param_grid = {'C': [0.1, 10, 100], 'gamma': [1, 0.1, 0.01], 'kernel': ['sigmoid', 'linear', 'poly', 'rbf']}
    #svc_param_grid = {'C': [0.1, 100], 'gamma': [1, 0.1, 0.01], 'kernel': ['sigmoid', 'poly', 'rbf']}
    svc_param_grid = {'C': [1], 
     #          'gamma': [1,0.1,0.01,0.001,0.0001], 
               'kernel': [ 'sigmoid']}
    svc_clf = SVC()
   # gamma='scale'
    svc_search = HalvingGridSearchCV(svc_clf, svc_param_grid, factor = 2, scoring = 'accuracy')
    svc_search.fit(X_train, y_train)
    
    svc_model = svc_search.best_params_
    svc_model_est = svc_search.best_estimator_
    
    y_pred = svc_model_est.predict(X_test)
    
    log_loss = metrics.log_loss(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred, average='weighted')
    recall = metrics.recall_score(y_test, y_pred, average='weighted')
    f1_score = metrics.f1_score(y_test, y_pred, average='macro')
    print (svc_model)
    return accuracy, precision, recall, f1_score, log_loss, y_pred, svc_model_est  

# Activate algorithm functions

#### On Lemmatized data

In [ ]:
# Lemmatized, without stopwords  

#Random Forest
acc_rf_lemma, precision_rf_lemma, recall_rf_lemma, f1_score_rf_lemma, log_loss_rf_lemma, y_pred_rf_lemma, berf_lemma  = rf_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)

report_rf_lemma = classification_report(y_pred_rf_lemma, y_test_lemma)
matrix_rf_lemma = confusion_matrix(y_pred_rf_lemma, y_test_lemma)
print('Classification Report - RF\n', report_rf_lemma)
print()
print('Confusion Matrix - RF\n', matrix_rf_lemma)

#Logistic Regression
acc_lr_lemma, precision_lr_lemma, recall_lr_lemma, f1_score_lr_lemma, log_loss_lr_lemma,  y_pred_lr_lemma, belr_lemma = lr_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)
print()


report_lr_lemma = classification_report(y_pred_lr_lemma, y_test_lemma)
matrix_lr_lemma = confusion_matrix(y_pred_lr_lemma, y_test_lemma)
print('Classification Report - Logistic Regression\n', report_lr_lemma)
print()
print('Confusion Matrix - Logistic Regression\n', matrix_lr_lemma)

# SVC 
acc_svc_lemma, precision_svc_lemma, recall_svc_lemma, f1_score_svc_lemma, log_loss_svc_lemma, y_pred_svc_lemma, besvc_lemma = svc_clf(X_train_lemma, X_test_lemma, y_train_lemma, y_test_lemma)

report_svc_lemma = classification_report(y_pred_svc_lemma, y_test_lemma)
matrix_svc_lemma = confusion_matrix(y_pred_svc_lemma, y_test_lemma)

print('Classification Report - SVC\n', report_svc_lemma)
print()
print('Confusion Matrix - SVC\n', matrix_svc_lemma)

RandomForestClassifier(max_depth=10, min_samples_split=3, n_estimators=10,
                       random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       1.00      0.98      0.99       478
           1       0.96      1.00      0.98       272

    accuracy                           0.99       750
   macro avg       0.98      0.99      0.98       750
weighted avg       0.99      0.99      0.99       750


Confusion Matrix - RF
 [[467  11]
 [  0 272]]
LogisticRegression(C=0.1, random_state=0, solver='sag')

Classification Report - Logistic Regression
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       467
           1       1.00      1.00      1.00       283

    accuracy                           1.00       750
   macro avg       1.00      1.00      1.00       750
weighted avg       1.00      1.00      1.00       750


Confusion Matrix - Logistic Regression
 [[467   0]
 [ 

In [ ]:
# Creating DataFrame for Evaluations
result_lemma = pd.DataFrame(columns = ['Model Name', 'Accuracy', 'Precision', 'Recall', 'F1_score', 'Log_loss'])
result_lemma.head()

,Model Name,Accuracy,Precision,Recall,F1_score,Log_loss


In [ ]:
result_lemma = res('RandomForest Classifier',acc_rf_lemma, precision_rf_lemma, recall_rf_lemma, f1_score_rf_lemma, log_loss_rf_lemma, y_pred_rf_lemma, berf_lemma)
result_lemma = res('Logistic Regression', acc_lr_lemma, precision_lr_lemma, recall_lr_lemma, f1_score_lr_lemma, log_loss_lr_lemma,  y_pred_lr_lemma, belr_lemma)
result_lemma = res('SVC Classifier', acc_svc_lemma, precision_svc_lemma, recall_svc_lemma, f1_score_svc_lemma, log_loss_svc_lemma, y_pred_svc_lemma, besvc_lemma)

## Lemmatized with stopwords

In [ ]:
# Lemmatized, with stopwords

#Random Forest
acc_rf_sw_lemma, precision_rf_sw_lemma, recall_rf_sw_lemma, f1_score_rf_sw_lemma, log_loss_rf_sw_lemma, y_pred_rf_sw_lemma, berf_sw_lemma  = rf_clf(X_train_sw_lemma, X_test_sw_lemma, y_train_sw_lemma, y_test_sw_lemma)

report_rf_sw_lemma = classification_report(y_pred_rf_sw_lemma, y_test_sw_lemma)
matrix_rf_sw_lemma = confusion_matrix(y_pred_rf_sw_lemma, y_test_sw_lemma)
print('Classification Report - RF\n', report_rf_sw_lemma)
print()
print('Confusion Matrix - RF\n', matrix_rf_sw_lemma)
print()

#Logistic Regression
acc_lr_sw_lemma, precision_lr_sw_lemma, recall_lr_sw_lemma, f1_score_lr_sw_lemma, log_loss_lr_sw_lemma,  y_pred_lr_sw_lemma, belr_sw_lemma = lr_clf(X_train_sw_lemma, X_test_sw_lemma, y_train_sw_lemma, y_test_sw_lemma)
print()


report_lr_sw_lemma = classification_report(y_pred_lr_sw_lemma, y_test_sw_lemma)
matrix_lr_sw_lemma = confusion_matrix(y_pred_lr_sw_lemma, y_test_sw_lemma)
print('Classification Report - Logistic Regression\n', report_lr_sw_lemma)
print()
print('Confusion Matrix - Logistic Regression\n', matrix_lr_sw_lemma)


# SVC 
acc_svc_sw_lemma, precision_svc_sw_lemma, recall_svc_sw_lemma, f1_score_svc_sw_lemma, log_loss_svc_sw_lemma, y_pred_svc_sw_lemma, besvc_sw_lemma = svc_clf(X_train_sw_lemma, X_test_sw_lemma, y_train_sw_lemma, y_test_sw_lemma)

report_svc_sw_lemma = classification_report(y_pred_svc_sw_lemma, y_test_sw_lemma)
matrix_svc_sw_lemma = confusion_matrix(y_pred_svc_sw_lemma, y_test_sw_lemma)

print('Classification Report - SVC\n', report_svc_sw_lemma)
print()
print('Confusion Matrix - SVC\n', matrix_svc_sw_lemma)

RandomForestClassifier(max_depth=10, min_samples_split=3, n_estimators=20,
                       random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       467
           1       1.00      1.00      1.00       283

    accuracy                           1.00       750
   macro avg       1.00      1.00      1.00       750
weighted avg       1.00      1.00      1.00       750


Confusion Matrix - RF
 [[467   0]
 [  0 283]]

LogisticRegression(C=0.01, random_state=0, solver='sag')
log loss 9.992007221626413e-16 
acc 1.0

Classification Report - SVC
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       469
           1       0.98      0.99      0.99       281

    accuracy                           0.99       750
   macro avg       0.99      0.99      0.99       750
weighted avg       0.99      0.99      0.99       750


Confusion Matrix - SVC
 [[46

In [ ]:
# Creating DataFrame for Evaluations
result_sw_lemma = pd.DataFrame(columns = ['Model Name', 'Accuracy', 'Precision', 'Recall', 'F1_score', 'Log_loss'])
result_sw_lemma.head()

In [ ]:
result_sw_lemma = res('RandomForest Classifier', acc_rf_sw_lemma, precision_rf_sw_lemma, recall_rf_sw_lemma, f1_score_rf_sw_lemma, log_loss_rf_sw_lemma, y_pred_rf_sw_lemma, berf_sw_lemma)
result_sw_lemma = res('Logistic Regression', acc_lr_sw_lemma, precision_lr_sw_lemma, recall_lr_sw_lemma, f1_score_lr_sw_lemma, log_loss_lr_sw_lemma,  y_pred_lr_sw_lemma, belr_sw_lemma)
result_sw_lemma = res('SVC Classifier', acc_svc_sw_lemma, precision_svc_sw_lemma, recall_svc_sw_lemma, f1_score_svc_sw_lemma, log_loss_svc_sw_lemma, y_pred_svc_sw_lemma, besvc_sw_lemma)

# With stopwords, not lemmatized nor stemmed

In [ ]:
# With stopwords, not lemmatized, nor stemmed

#Random Forest
acc_rf_sw, precision_rf_sw, recall_rf_sw, f1_score_rf_sw, log_loss_rf_sw, y_pred_rf_sw, berf_sw  = rf_clf(X_train_sw, X_test_sw, y_train_sw, y_test_sw)

report_rf_sw = classification_report(y_pred_rf_sw, y_test_sw)
matrix_rf_sw = confusion_matrix(y_pred_rf_sw, y_test_sw)
print('Classification Report - RF\n', report_rf_sw)
print()
print('Confusion Matrix - RF\n', matrix_rf_sw)
print()

#Logistic Regression
acc_lr_sw, precision_lr_sw, recall_lr_sw, f1_score_lr_sw, log_loss_lr_sw,  y_pred_lr_sw, belr_sw = lr_clf(X_train_sw, X_test_sw, y_train_sw, y_test_sw)
print()
report_lr_sw = classification_report(y_pred_lr_sw, y_test_sw)
matrix_lr_sw = confusion_matrix(y_pred_lr_sw, y_test_sw)
print('Classification Report - Logistic Regression\n', report_lr_sw)
print()
print('Confusion Matrix - Logistic Regression\n', matrix_lr_sw)


# SVC 
acc_svc_sw, precision_svc_sw, recall_svc_sw, f1_score_svc_sw, log_loss_svc_sw, y_pred_svc_sw, besvc_sw= svc_clf(X_train_sw, X_test_sw, y_train_sw, y_test_sw)

report_svc_sw = classification_report(y_pred_svc_sw, y_test_sw)
matrix_svc_sw = confusion_matrix(y_pred_svc_sw, y_test_sw)

print('Classification Report - SVC\n', report_svc_sw)
print()
print('Confusion Matrix - SVC\n', matrix_svc_sw)

RandomForestClassifier(max_depth=10, min_samples_split=3, n_estimators=20,
                       random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       468
           1       1.00      1.00      1.00       282

    accuracy                           1.00       750
   macro avg       1.00      1.00      1.00       750
weighted avg       1.00      1.00      1.00       750


Confusion Matrix - RF
 [[467   1]
 [  0 282]]

LogisticRegression(C=0.1, random_state=0, solver='sag')
log loss 9.992007221626413e-16 
acc 1.0

Classification Report - SVC
               precision    recall  f1-score   support

           0       0.99      0.99      0.99       468
           1       0.98      0.98      0.98       282

    accuracy                           0.99       750
   macro avg       0.98      0.98      0.98       750
weighted avg       0.99      0.99      0.99       750


Confusion Matrix - SVC
 [[462

In [ ]:
# Creating DataFrame for Evaluations
result_sw = pd.DataFrame(columns = ['Model Name', 'Accuracy', 'Precision', 'Recall', 'F1_score', 'Log_loss'])
result_sw.head()

In [ ]:
result_sw = res('RandomForest Classifier', acc_rf_sw, precision_rf_sw, recall_rf_sw, f1_score_rf_sw, log_loss_rf_sw, y_pred_rf_sw, berf_sw)
result_sw = res('Logistic Regression',acc_lr_sw, precision_lr_sw, recall_lr_sw, f1_score_lr_sw, log_loss_lr_sw,  y_pred_lr_sw, belr_sw)
result_sw = res('SVC Classifier', acc_svc_sw, precision_svc_sw, recall_svc_sw, f1_score_svc_sw, log_loss_svc_sw, y_pred_svc_sw, besvc_sw)

### Without stopwords, not lemmatized nor stemmed

In [ ]:
# Without stopwords, not lemmatized, nor stemmed

#Random Forest
acc_rf_out, precision_rf_out, recall_rf_out, f1_score_rf_out, log_loss_rf_out, y_pred_rf_out, berf_out  = rf_clf(X_train_out, X_test_out, y_train_out, y_test_out)

report_rf_out = classification_report(y_pred_rf_out, y_test_out)
matrix_rf_out = confusion_matrix(y_pred_rf_out, y_test_out)
print('Classification Report - RF\n', report_rf_out)
print()
print('Confusion Matrix - RF\n', matrix_rf_out)
print()

#Logistic Regression
acc_lr_out, precision_lr_out, recall_lr_out, f1_score_lr_out, log_loss_lr_out,  y_pred_lr_out, belr_out = lr_clf(X_train_out, X_test_out, y_train_out, y_test_out)
print()
report_lr_out = classification_report(y_pred_lr_out, y_test_out)
matrix_lr_out = confusion_matrix(y_pred_lr_out, y_test_out)
print('Classification Report - Logistic Regression\n', report_lr_out)
print()
print('Confusion Matrix - Logistic Regression\n', matrix_lr_out)


# SVC 
acc_svc_out, precision_svc_out, recall_svc_out, f1_score_svc_out, log_loss_svc_out, y_pred_svc_out, besvc_out = svc_clf(X_train_out, X_test_out, y_train_out, y_test_out)

report_svc_out = classification_report(y_pred_svc_out, y_test_out)
matrix_svc_out = confusion_matrix(y_pred_svc_out, y_test_out)

print('Classification Report - SVC\n', report_svc_out)
print()
print('Confusion Matrix - SVC\n', matrix_svc_out)

RandomForestClassifier(max_depth=10, min_samples_split=3, n_estimators=20,
                       random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       468
           1       1.00      1.00      1.00       282

    accuracy                           1.00       750
   macro avg       1.00      1.00      1.00       750
weighted avg       1.00      1.00      1.00       750


Confusion Matrix - RF
 [[467   1]
 [  0 282]]

LogisticRegression(C=0.01, random_state=0, solver='sag')
log loss 9.992007221626413e-16 
acc 1.0

Classification Report - SVC
               precision    recall  f1-score   support

           0       0.98      0.97      0.98       470
           1       0.96      0.97      0.96       280

    accuracy                           0.97       750
   macro avg       0.97      0.97      0.97       750
weighted avg       0.97      0.97      0.97       750


Confusion Matrix - SVC
 [[45

In [ ]:
# Creating DataFrame for Evaluations
result_out = pd.DataFrame(columns = ['Model Name', 'Accuracy', 'Precision', 'Recall', 'F1_score', 'Log_loss'])
result_out.head()

In [ ]:
result_out = res('RandomForest Classifier', acc_rf_out, precision_rf_out, recall_rf_out, f1_score_rf_out, log_loss_rf_out, y_pred_rf_out, berf_out)
result_out = res('Logistic Regression', acc_lr_out, precision_lr_out, recall_lr_out, f1_score_lr_out, log_loss_lr_out,  y_pred_lr_out, belr_out)
result_out = res('SVC Classifier', acc_svc_out, precision_svc_out, recall_svc_out, f1_score_svc_out, log_loss_svc_out, y_pred_svc_out, besvc_out)

# Stemmed data with stopwords

In [ ]:
# Stemmed with stopwords

#Random Forest
acc_rf_sw_stem, precision_rf_sw_stem, recall_rf_sw_stem, f1_score_rf_sw_stem, log_loss_rf_sw_stem, y_pred_rf_sw_stem, berf_sw_stem  = rf_clf(X_train_sw_stem, X_test_sw_stem, y_train_sw_stem, y_test_sw_stem)

report_rf_sw_stem = classification_report(y_pred_rf_sw_stem, y_test_sw_stem)
matrix_rf_sw_stem = confusion_matrix(y_pred_rf_sw_stem, y_test_sw_stem)
print('Classification Report - RF\n', report_rf_sw_stem)
print()
print('Confusion Matrix - RF\n', matrix_rf_sw_stem)
print()


#Logistic Regression
acc_lr_sw_stem, precision_lr_sw_stem, recall_lr_sw_stem, f1_score_lr_sw_stem, log_loss_lr_sw_stem,  y_pred_lr_sw_stem, belr_sw_stem = lr_clf(X_train_sw_stem, X_test_sw_stem, y_train_sw_stem, y_test_sw_stem)
print()
report_lr_sw_stem = classification_report(y_pred_lr_sw_stem, y_test_sw_stem)
matrix_lr_sw_stem = confusion_matrix(y_pred_lr_sw_stem, y_test_sw_stem)
print('Classification Report - Logistic Regression\n', report_lr_sw_stem)
print()
print('Confusion Matrix - Logistic Regression\n', matrix_lr_sw_stem)


# SVC 
acc_svc_sw_stem, precision_svc_sw_stem, recall_svc_sw_stem, f1_score_svc_sw_stem, log_loss_svc_sw_stem, y_pred_svc_sw_stem, besvc_sw_stem = svc_clf(X_train_sw_stem, X_test_sw_stem, y_train_sw_stem, y_test_sw_stem)

report_svc_sw_stem = classification_report(y_pred_svc_sw_stem, y_test_sw_stem)
matrix_svc_sw_stem = confusion_matrix(y_pred_svc_sw_stem, y_test_sw_stem)

print('Classification Report - SVC\n', report_svc_sw_stem)
print()
print('Confusion Matrix - SVC\n', matrix_svc_sw_stem)

RandomForestClassifier(max_depth=10, min_samples_split=3, n_estimators=20,
                       random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       468
           1       1.00      1.00      1.00       282

    accuracy                           1.00       750
   macro avg       1.00      1.00      1.00       750
weighted avg       1.00      1.00      1.00       750


Confusion Matrix - RF
 [[467   1]
 [  0 282]]

LogisticRegression(C=0.1, random_state=0, solver='sag')
log loss 9.992007221626413e-16 
acc 1.0

Classification Report - SVC
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       468
           1       0.99      1.00      0.99       282

    accuracy                           1.00       750
   macro avg       1.00      1.00      1.00       750
weighted avg       1.00      1.00      1.00       750


Confusion Matrix - SVC
 [[466

In [ ]:
# Creating DataFrame for Evaluations
result_sw_stem = pd.DataFrame(columns = ['Model Name', 'Accuracy', 'Precision', 'Recall', 'F1_score', 'Log_loss'])
result_sw_stem.head()

In [ ]:
result_sw_stem = res('RandomForest Classifier', acc_rf_sw_stem, precision_rf_sw_stem, recall_rf_sw_stem, f1_score_rf_sw_stem, log_loss_rf_sw_stem, y_pred_rf_sw_stem, berf_sw_stem)
result_sw_stem = res('Logistic Regression', acc_lr_sw_stem, precision_lr_sw_stem, recall_lr_sw_stem, f1_score_lr_sw_stem, log_loss_lr_sw_stem,  y_pred_lr_sw_stem, belr_sw_stem)
result_sw_stem = res('SVC Classifier', acc_svc_sw_stem, precision_svc_sw_stem, recall_svc_sw_stem, f1_score_svc_sw_stem, log_loss_svc_sw_stem, y_pred_svc_sw_stem, besvc_sw_stem)

# Stemmed without stopwords


In [ ]:
# Stemmed with stopwords

#Random Forest
acc_rf_stem, precision_rf_stem, recall_rf_stem, f1_score_rf_stem, log_loss_rf_stem, y_pred_rf_stem, berf_stem  = rf_clf(X_train_stem, X_test_stem, y_train_stem, y_test_stem)

report_rf_stem = classification_report(y_pred_rf_stem, y_test_stem)
matrix_rf_stem = confusion_matrix(y_pred_rf_stem, y_test_stem)
print('Classification Report - RF\n', report_rf_stem)
print()
print('Confusion Matrix - RF\n', matrix_rf_stem)
print()

#Logistic Regression
acc_lr_stem, precision_lr_stem, recall_lr_stem, f1_score_lr_stem, log_loss_lr_stem,  y_pred_lr_stem, belr_stem = lr_clf(X_train_stem, X_test_stem, y_train_stem, y_test_stem)
print()
report_lr_stem = classification_report(y_pred_lr_stem, y_test_stem)
matrix_lr_stem = confusion_matrix(y_pred_lr_stem, y_test_stem)
print('Classification Report - Logistic Regression\n', report_lr_stem)
print()
print('Confusion Matrix - Logistic Regression\n', matrix_lr_stem)

# SVC 
acc_svc_stem, precision_svc_stem, recall_svc_stem, f1_score_svc_stem, log_loss_svc_stem, y_pred_svc_stem, besvc_stem = svc_clf(X_train_stem, X_test_stem, y_train_stem, y_test_stem)

report_svc_stem = classification_report(y_pred_svc_stem, y_test_stem)
matrix_svc_stem = confusion_matrix(y_pred_svc_stem, y_test_stem)

print('Classification Report - SVC\n', report_svc_stem)
print()
print('Confusion Matrix - SVC\n', matrix_svc_stem)

RandomForestClassifier(max_depth=10, min_samples_split=3, n_estimators=20,
                       random_state=0)
Classification Report - RF
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       467
           1       1.00      1.00      1.00       283

    accuracy                           1.00       750
   macro avg       1.00      1.00      1.00       750
weighted avg       1.00      1.00      1.00       750


Confusion Matrix - RF
 [[467   0]
 [  0 283]]

LogisticRegression(C=0.1, random_state=0, solver='sag')
log loss 9.992007221626413e-16 
acc 1.0

Classification Report - SVC
               precision    recall  f1-score   support

           0       0.99      0.97      0.98       475
           1       0.95      0.98      0.97       275

    accuracy                           0.98       750
   macro avg       0.97      0.98      0.97       750
weighted avg       0.98      0.98      0.98       750


Confusion Matrix - SVC
 [[462

In [ ]:
# Creating DataFrame for Evaluations
result_stem = pd.DataFrame(columns = ['Model Name', 'Accuracy', 'Precision', 'Recall', 'F1_score', 'Log_loss'])
result_stem.head()

In [ ]:
result_stem = res('RandomForest Classifier', acc_rf_stem, precision_rf_stem, recall_rf_stem, f1_score_rf_stem, log_loss_rf_stem, y_pred_rf_stem, berf_stem)
result_stem = res('Logistic Regression', acc_lr_stem, precision_lr_stem, recall_lr_stem, f1_score_lr_stem, log_loss_lr_stem,  y_pred_lr_stem, belr_stem)
result_stem = res('SVC Classifier', acc_svc_stem, precision_svc_stem, recall_svc_stem, f1_score_svc_stem, log_loss_svc_stem, y_pred_svc_stem, besvc_stem)

# Cosine Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#cosine = cosine_similarity(X_train_lemma, y_train_lemma, dense_output=True)

In [ ]:
cosine_similarity(arr_q1_lemma , arr_q2_lemma, dense_output=True)

array([[0.77353704, 0.6493094 , 0.7844405 , ..., 0.61083734, 0.6324987 ,
        0.8206404 ],
       [0.33382827, 0.8227184 , 0.5351572 , ..., 0.5696451 , 0.4045978 ,
        0.5432366 ],
       [0.57329106, 0.65007234, 0.86115694, ..., 0.77194357, 0.52491057,
        0.7496874 ],
       ...,
       [0.5479884 , 0.5654626 , 0.74211514, ..., 0.8167627 , 0.58999306,
        0.8312438 ],
       [0.63509995, 0.54801077, 0.78231716, ..., 0.63727254, 0.88014674,
        0.81301326],
       [0.6825827 , 0.6387254 , 0.84284997, ..., 0.6468858 , 0.6542733 ,
        0.8440794 ]], dtype=float32)

In [ ]:
cosine_similarity(arr_q1_lemma , arr_q2_lemma)
cosine_similarity(arr_q1_sw_lemma , arr_q2_sw_lemma)
cosine_similarity(arr_q1_stem , arr_q2_stem)
cosine_similarity(arr_q1_sw_stem , arr_q2_sw_stem)
cosine_similarity(arr_q1_sw , arr_q2_sw)
cosine_similarity(arr_q1_out , arr_q2_out)